# Noise Floor Estimation

Determine noise floor for timeseries data

Can use to set appropriate Gaussian noise augmentation std without swamping any signals

In [ ]:
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt

from signal_diffusion.config import load_settings
cfg = load_settings()

In [ ]:
ds_path = "/data/data/signal-diffusion/processed/reweighted_timeseries_meta_dataset_n2048_fs125"
ds = load_dataset(ds_path, split="train").with_format("numpy")

In [ ]:
signals = ds[:100]["signal"]
signals.shape

In [ ]:
nsamp = signals.shape[-1]
X = np.fft.rfft(signals, axis=-1) / np.sqrt(nsamp)
plt.plot(np.arange(1025) * 125 / 2048, 20 * np.log10(np.abs(X).mean(axis=0).T))
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (dB)")
plt.title("Average Power Spectral Density")
plt.grid()
plt.xlim(-1, 62.5)
plt.ylim(-80, 30)

N0 = 10 ** (-60 / 20)  # dB to linear
print(f"Chosen N0: {N0:.2e} linear scale, {20 * np.log10(N0):.2f} dB scale")
plt.hlines(20 * np.log10(N0), 0, 62.5, colors="r",
           linestyles="dashed", label="Chosen noise floor")

aug_noise = np.random.normal(0, N0, size=signals.shape[1:])
X_aug = np.fft.rfft(aug_noise) / np.sqrt(nsamp)
plt.plot(np.arange(1025) * 125 / 2048, 20*np.log10(np.abs(X_aug).mean(axis=0)),
         label="Augmentation noise PSD", color="k", alpha=0.5,
         )

plt.legend();

## Est. noise floor

Based on average for lower channels, noise floor ~~ -60 dB

This corresponds to noise std = 0.001